## ML-Assignment 2
Muhammed Talha Gedikli - 19069608

## Import Libraries and Load Data

In [1]:
# Load important libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.datasets import make_regression

from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import Ridge, Lasso

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.svm import LinearSVC
from sklearn.svm import SVC #note that this is non-linear SVM and its default is RBF
from sklearn.datasets import make_regression #function used for generating synthetic datasets
from sklearn.datasets import make_classification, make_blobs

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error, r2_score


# Import data
train_data = pd.read_excel("./asg1_train_test_data/asg1_traindata.xlsx")
test_data = pd.read_excel("./asg1_train_test_data/asg1_testdata_without_prices.xlsx")

test_data_without_prices = pd.read_excel("./asg1_train_test_data/asg1_testdata_without_prices.xlsx")

features_X = ["Year", "Type", "Shift", "km", "Power", "Engine", "Seller"]
features_y = ["Price"]


# Split data into features - prices
X_data = train_data[features_X]
y_data = train_data[features_y]

X_data_without_prices = test_data_without_prices


# Split data into train and test sets
# default is 75% / 25% train-test split
# X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=735)

# X_test = test_data[features_X]
# y_test = test_data[features_y] Y values of test is not given
# y_test = pd.read_csv("./Result.txt", sep=" ", header=None) S

# default 'n' value for knn or cv
N = 5

In [2]:
''' 
# Deal with categorical data
from sklearn.preprocessing import OneHotEncoder

# Find the columns to deal with
object_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(X_test[object_cols]))

# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")
OH_cols_train.columns = OH_encoder.get_feature_names_out(object_cols)
OH_cols_test.columns = OH_encoder.get_feature_names_out(object_cols)

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)
'''



' \n# Deal with categorical data\nfrom sklearn.preprocessing import OneHotEncoder\n\n# Find the columns to deal with\nobject_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]\n\n# Apply one-hot encoder to each column with categorical data\nOH_encoder = OneHotEncoder(handle_unknown=\'ignore\', sparse=False)\nOH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))\nOH_cols_test = pd.DataFrame(OH_encoder.fit_transform(X_test[object_cols]))\n\n# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")\nOH_cols_train.columns = OH_encoder.get_feature_names_out(object_cols)\nOH_cols_test.columns = OH_encoder.get_feature_names_out(object_cols)\n\n# One-hot encoding removed index; put it back\nOH_cols_train.index = X_train.index\nOH_cols_test.index = X_test.index\n\n# Remove categorical columns (will replace with one-hot encoding)\nnum_X_train = X_train.drop(object_cols, axis=1)\nnum_X_test =

## Handle With Categorical Data and Do Feature Scaling

In [3]:
# Deal with categorical data
from sklearn.preprocessing import OneHotEncoder

# Find the columns to deal with
object_cols = [col for col in X_data.columns if X_data[col].dtype == "object"]

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(X_data[object_cols]))
OH_X_without_prices_cols = pd.DataFrame(OH_encoder.fit_transform(X_data_without_prices[object_cols]))


# Give them column names (for example "Shift_Auto, Shift_Manual") instead of just integer column names (like "0, 1, 2")
OH_cols.columns = OH_encoder.get_feature_names_out(object_cols)
OH_X_without_prices_cols.columns = OH_encoder.get_feature_names_out(object_cols)

# One-hot encoding removed index; put it back
OH_cols.index = X_data.index
OH_X_without_prices_cols.index = X_data_without_prices.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_data.drop(object_cols, axis=1)
num_X_without_prices = X_data_without_prices.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_data = pd.concat([num_X_train, OH_cols], axis=1)
OH_X_data_without_prices = pd.concat([num_X_without_prices, OH_X_without_prices_cols], axis=1)

# Split data into train and test sets
# default is 75% / 25% train-test split
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=735)

# Feature scaling
from sklearn.preprocessing import MinMaxScaler

# Set scaler
scaler = MinMaxScaler()

##X_train, X_test, y_train, y_test = train_test_split(X_house, y_house)

# Scale data
X_train_scaled = scaler.fit_transform(X_train) # we use X_train to adjust/fit the scaler
X_test_scaled  = scaler.transform(X_test) #use the same fit found above to transform X_test# Scale data

OH_X_data_scaled = scaler.fit_transform(OH_X_data) # we use X_train to adjust/fit the scaler

## Lineer Regression

In [4]:
# Import required libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score

# Prepare Data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=5)

# Make the model
linr = LinearRegression().fit(X_train, y_train)

# Evalueate and save scores
linr_scores = {
     "r2_train": linr.score(X_train, y_train),
     "r2_test": linr.score(X_test, y_test),
     "r2_cv": cross_val_score(linr, X_train, y_train, cv=N, scoring="r2").mean()
}

# Print the scores
for k, v in linr_scores.items():
     print(k + ": " + str(v))

R-squared score (training): 0.862
R-squared score (test): 0.890

CV Score (test): 0.864



## Polynomial Regression

In [22]:
# Import required libraries
from sklearn.datasets import make_friedman1
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Define variables to evaluate best model
polyr_degree = 0
polyr_score = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True)

# Loop through different variables to get best model
for this_degree in [1, 2, 3, 4, 5]:
     # Make polynomial features with 'this_degree'
     # I used 'OH_X_data_scaled' to evalueate faster (it will take some time)
     poly = PolynomialFeatures(degree=this_degree)
     X_F1_poly = poly.fit_transform(OH_X_data_scaled)
     
     # Preapare polynomial data for regression
     X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y_data, random_state = 1)
     
     # Make the model with data
     polyr = LinearRegression().fit(X_train, y_train)

     # Get the performance score
     this_score = cross_val_score(polyr, X_train, y_train, cv=N, scoring='r2').mean()
     this_score = polyr.score(X_test, y_test)
     
     # I make test with cross validation score, if new CV score is better than old one, just get new ones
     if (this_score > polyr_score):
          polyr_score = this_score
          polyr_degree = this_degree

# Make the final model with the best variables
poly = PolynomialFeatures(degree=polyr_degree)
X_F1_poly = poly.fit_transform(OH_X_data_scaled)

# Prepare data
X_train, X_test, y_train, y_test = train_test_split(X_F1_poly, y_data, random_state = 1)

# Make model
polyr = LinearRegression().fit(X_train, y_train)


# Get and save the r2 values of the model
polyr_scores = {
     'optimal_degree': polyr_degree,
     'r2_train': polyr.score(X_train, y_train),
     'r2_test': polyr.score(X_test, y_test),
     'r2_cv': cross_val_score(polyr, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print scores and variables for the model
for k, v in polyr_scores.items():
     print(k + ": " + str(v))


optimal_degree: 1
r2_train: 0.8920660929409385
r2_test: 0.904752423958178
r2_cv: 0.8887804925359737


## Ridge Regression

In [6]:
# Import required libraries
from sklearn.linear_model import Ridge

# Preapare data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state = 1)

# for this_alpha in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40]:
# Define variables to evaluate best model
ridgr_alpha = 0
ridgr_cv = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True)

# Loop through different variables to get best model
for this_a in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.35, 1.37, 1.43, 1.45]:
     # Make model with 'this_a' alpha
     ridgr = Ridge(alpha=this_a, random_state=1).fit(X_train, y_train)

     # Calculate the score with the model just made
     this_cv = cross_val_score(ridgr, X_train, y_train, cv=N, scoring='r2').mean()
     
     # if new score is better than old one, just assign new values to old ones
     if (this_cv > ridgr_cv):
          ridgr_cv = this_cv
          ridgr_alpha = this_a

# Make the final model with the best variables
ridgr = Ridge(alpha=ridgr_alpha, random_state=1).fit(X_train, y_train)

# Get and save the r2 values of the model
ridgr_scores = {
     'optimal_alpha': ridgr_alpha,
     'r2_train': ridgr.score(X_train, y_train),
     'r2_test': ridgr.score(X_test, y_test),
     'r2_cv': cross_val_score(ridgr, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print scores and variables for the model
for k, v in ridgr_scores.items():
     print(k + ": " + str(v))

optimal_alpha: 1.37
r2_train: 0.862063615440508
r2_test: 0.8899703103623747
r2_cv: 0.8637501230071969


## Lasso Regression

In [7]:
# Import required libraries
from sklearn.linear_model import Lasso

# Preapare data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state = 1)

# Define variables to evaluate best model
lassr_alpha = 0
lassr_cv = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True)

# Loop through different variables to get best model
for this_a in [0, 0.0001, 0.001, 0.01, 0.1]:
     # Make model with 'this_a' alpha
     lassr = Lasso(alpha=this_a, random_state=1).fit(X_train, y_train)

     # Calculate the score with the model just made
     this_cv = cross_val_score(lassr, X_train, y_train, cv=N, scoring='r2').mean()

     # if new score is better than old one, just assign new values to old ones
     if (this_cv > lassr_cv):
          lassr_cv = this_cv
          lassr_alpha = this_a

# Make the final model with the best variables
lassr = Lasso(alpha=lassr_alpha, random_state=1).fit(X_train, y_train)


# Get and save the r2 values of the model
lassr_scores = {
     'optimal_alpha': lassr_alpha,
     'r2_train': lassr.score(X_train, y_train),
     'r2_test': lassr.score(X_test, y_test),
     'r2_cv': cross_val_score(lassr, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print scores and variables for the model
for k, v in lassr_scores.items():
     print(k + ": " + str(v))

C:\Users\talha\AppData\Local\Temp\ipykernel_20344\2871493436.py:11: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lassr = Lasso(alpha=this_a, random_state=1).fit(X_train, y_train)
c:\Users\talha\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\talha\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.363e+10, tolerance: 3.549e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model =

0
0.8696459237754061


## KNN

In [28]:
# Import required libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Prepare data
X_train, X_test, y_train, y_test = train_test_split(OH_X_data, y_data, random_state=0)

# Define variables to evaluate best model
knn_n = 0
knn_score = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True)

# Loop through different variables to get best model
for this_n in [1, 2, 3, 4, 5]:
     # Make model with 'this_n' alpha
     knn = KNeighborsClassifier(n_neighbors = this_n).fit(X_train, y_train)

     # Calculate the score with the model just made
     this_cv = cross_val_score(knn, X_train, y_train, cv=kf5, scoring='r2').mean()

     # if new score is better than old one, just assign new values to old ones
     if (this_cv > knn_score):
          knn_score = this_cv
          knn_n = this_n

# Make model with best variable
knn = KNeighborsClassifier(n_neighbors = knn_n).fit(X_train, y_train)

# Get and save r2 scores for the model
knn_scores = {
     'optimal_n': knn_n,
     'r2_train': knn.score(X_train, y_train),
     'r2_test': knn.score(X_test, y_test),
     'r2_cv': cross_val_score(knn, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print scores and variables for the model
for k, v in knn_scores.items():
     print(k + ": " + str(v))

c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or Fa

optimal_nneighbors: 100
r2_train: 0.05241379310344828
r2_test: 0.008275862068965517
r2_cv: 0.5503478870602656


c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\talha\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode

## Linear SVM

In [29]:
# Import required libraries
from sklearn.svm import LinearSVC

# Prepare data (with scaled data)
X_train, X_test, y_train, y_test = train_test_split(OH_X_data_scaled, y_data.values.ravel(), random_state=1)

# Define variables to evaluate best model
linsvc_c = 1
linsvc_cv = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True) 

# Loop through different variables to get best model
for this_c in [4, 5, 10, 20]:
    # Make model with 'this_c'
    linsvc = LinearSVC(C=this_c).fit(X_train, y_train)

    # Calculate the score with the model just made
    this_cv = cross_val_score(linsvc, X_train, y_train, cv=kf5, scoring='r2').mean()

    # if new score is better than old one, just assign new values to old ones
    if (this_cv > linsvc_cv):
        linsvc_cv = this_cv
        linsvc_c = this_c

# Make model with best variable
linsvc = LinearSVC(C=linsvc_c).fit(X_train, y_train)

# Get and save r2 scores for the model
linsvc_scores = {
    'optimal_c': linsvc_c,
    'r2_train': linsvc.score(X_train, y_train),
    'r2_test': linsvc.score(X_test, y_test),
    'r2_cv': cross_val_score(linsvc, X_train, y_train, cv=kf5, scoring="r2").mean()
}

# Print scores and variables for the model
for k, v in linsvc_scores.items():
     print(k + ": " + str(v))

c:\Users\talha\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\talha\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\talha\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\talha\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\talha\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\talha\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number 

optimal_c: 5
r2_train: 0.08735632183908046
r2_test: 0.02620689655172414
r2_cv: 0.6965658528753668


## Non-Linear SVM

In [12]:
# Import required libraries
from sklearn.svm import SVC

# Prepare data (with scaled data)
X_train, X_test, y_train, y_test = train_test_split(OH_X_data_scaled, y_data.values.ravel())

# Define variables to evaluate best model
svc_c = 1
svc_cv = 0

# Set kfold for evaluate best results for CV
kf5 = KFold(n_splits=N, shuffle=True)

# Loop through different variables to get best model
for this_c in [10, 1000, 10000]:
    # Make model with 'this_c' alpha
    svc = SVC(C=this_c, random_state=1).fit(X_train, y_train)

    # Calculate the score with the model just made
    this_cv = cross_val_score(svc, X_train, y_train, cv=kf5, scoring='r2').mean()

    # if new score is better than old one, just assign new values to old ones
    if (this_cv > svc_cv):
        svc_cv = this_cv
        svc_c = this_c

# Make model with best variable
svc = SVC(C=svc_c, random_state=1).fit(X_train, y_train)

# Get and save r2 scores for the model
svc_scores = {
    'optimal_c': svc_c,
    'r2_train': svc.score(X_train, y_train),
    'r2_test': svc.score(X_test, y_test),
    'r2_cv': cross_val_score(svc, X_train, y_train, cv=kf5, scoring='r2').mean()
}

# Print scores and variables for the model
for k, v in svc_scores.items():
     print(k + ": " + str(v))

optimal_c: 10000
r2_train: 0.608735632183908
r2_test: 0.038620689655172416
r2_cv: [0.86937986 0.87177931 0.73747353 0.89594431 0.72395207]


## Create excel file to export scores

In [47]:
# Create excel file includeing scores etc
table = pd.DataFrame(columns=["R2 Train", "R2 Test", "R2 CV"])
table.loc["Linear Regression"] = [linr_scores["r2_train"], linr_scores["r2_test"], linr_scores["r2_cv"].mean()]
table.loc["Polynomial Regression (degree: {})".format(polyr_scores['optimal_degree'])] = [
            polyr_scores["r2_train"], polyr_scores["r2_test"], polyr_scores["r2_cv"].mean()]
table.loc["Ridge Regression (alpha: {})".format(ridgr_scores['optimal_alpha'])] = [
            ridgr_scores["r2_train"], ridgr_scores["r2_test"], ridgr_scores["r2_cv"].mean()]
table.loc["Lasso Regression (alpha: {})".format(lassr_scores['optimal_alpha'])] = [
            lassr_scores["r2_train"], lassr_scores["r2_test"], lassr_scores["r2_cv"].mean()]
table.loc["KNN (n: {})".format(knn_scores['optimal_n'])] = [
            knn_scores["r2_train"], knn_scores["r2_test"], knn_scores["r2_cv"].mean()]
table.loc["Non Linear Svm (c: {})".format(svc_scores['optimal_c'])] = [
            svc_scores["r2_train"], svc_scores["r2_test"], svc_scores["r2_cv"].mean()]
table.loc["Linear SVM"] = [linsvc_scores["r2_train"], linsvc_scores["r2_test"], linsvc_scores["r2_cv"].mean()]

table.to_excel("Table.xlsx")
table

,R2 Train,R2 Test,R2 CV
Linear Regression,0.862290,0.889609,0.864140
Polynomial Regression (degree: 1),0.892066,0.904752,0.888780
Ridge Regression (alpha: 1.37),0.862064,0.889970,0.863750
Lasso Regression (alpha: 0),0.866812,0.874707,0.867394
KNN,0.052414,0.008276,0.550348
SVM (Non Linear),0.608736,0.038621,0.819706
SVM (Linear),0.087356,0.026207,0.696566


## Export Results

In [41]:
# Export Results
best_model = polyr
# price_predictions = best_model.predict(OH_X_data_without_prices)
# price_predictions_int = np.round_(price_predictions)
# np.savetxt('Resultmine.txt', price_predictions)
# price_predictions_int
best_model.coef_


array([[ 1.51648836e+03, -4.44818387e-02,  1.25394940e+01,
         8.80947869e+00, -1.38853321e+02, -3.46668629e+02,
         4.85521950e+02,  1.96688938e+03, -1.96688938e+03,
        -1.30648803e+02,  1.30648803e+02]])